<div style="text-align: center;">
    <h1>GARCH Hyperparameter Optimizer Example Test Case</h1>
</div>

# 1. Importing Necessary Modules

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from garch_optimizer.garch_optimizer import garch_parameter
from sklearn.metrics import mean_absolute_percentage_error
from arch import arch_model
import plotly.graph_objects as go
from pprint import pprint

# 2. Generating Sample Data

In [2]:
np.random.seed(42)
n = 500
returns = np.random.normal(0, 1, n)

volatility = np.zeros_like(returns)
volatility[0] = 1

for i in range(1, n):
    volatility[i] = np.sqrt(0.01 + 0.9 * volatility[i-1]**2 + 0.05 * returns[i-1]**2)

garch_returns = volatility * returns

df = pd.DataFrame(garch_returns, columns=["returns"])
df["date"] = pd.date_range(start="2022-01-01", periods=n)
df.set_index("date", inplace=True)
df.head()

,returns
date,
2022-01-01,0.496714
2022-01-02,-0.132787
2022-01-03,0.593990
2022-01-04,1.351922
2022-01-05,-0.213980


# 3. Running the GARCH Parameter Optimization

In [9]:
best_params = garch_parameter(
    data=df["returns"], 
    max_p=2, 
    max_q=2, 
    max_o=1, 
    max_lag=1, 
    n_trials=100
)

  0%|          | 0/100 [00:00<?, ?it/s]

Elapsed Time: 00:00:15.93


In [11]:
pprint(best_params)

{'error_dist': 'normal',
 'lags': 1,
 'mean_model': 'zero',
 'o': 0,
 'p': 1,
 'q': 1,
 'vol_model': 'egarch'}


# 4. Evaluating the Model

In [4]:
model = arch_model(df["returns"], mean=best_params['mean_model'], vol=best_params['vol_model'], 
                   p=best_params['p'], q=best_params['q'], o=best_params['o'], lags=best_params['lags'], 
                   rescale=True, dist=best_params['error_dist'])
model_fit = model.fit(disp='off')

df["Conditional Volatility"] = model_fit.conditional_volatility

In [5]:
true_volatility = df['returns'].ewm(20).std()
df['True Volatility'] = true_volatility
mape = mean_absolute_percentage_error(true_volatility[1:], df["Conditional Volatility"][1:])

print(f"MAPE between True and Predicted Volatility: {mape:.2f}%")

MAPE between True and Predicted Volatility: 0.09%


# 5. Visualizing the Results

In [6]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, 
    y=df["returns"], 
    mode='lines',
    name="Returns",
    line=dict(color='blue', width=2),
    opacity=0.6
))

fig.add_trace(go.Scatter(
    x=df.index, 
    y=df["Conditional Volatility"], 
    mode='lines',
    name="Conditional Volatility",
    line=dict(color='red', width=2)
))

fig.add_trace(go.Scatter(
    x=df.index, 
    y=df["True Volatility"], 
    mode='lines',
    name="True Volatility",
    line=dict(color='green', width=2)
))

fig.update_layout(
    title="True vs Predicted Conditional Volatility",
    xaxis_title="Date",
    yaxis_title="Value",
    legend=dict(orientation='h'),
    template="plotly",
    hovermode="x unified"
)

fig.show()
